In [1]:
import os
import glob
import pandas as pd
import dask.dataframe as dd
from dask.distributed import Client
import fastparquet
import mlfinlab as ml
from tqdm.notebook import tqdm

IDIR = 'data/clean_tick/'
ODIR = 'data/30min/'

freq = '30Min'

/home/dan/Apps/anaconda3/envs/financial-ml/lib/python3.6/site-packages/fastparquet/encoding.py:222: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  Numpy8 = numba.jitclass(spec8)(NumpyIO)
/home/dan/Apps/anaconda3/envs/financial-ml/lib/python3.6/site-packages/fastparquet/encoding.py:224: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  Numpy32 = numba.jitclass(spec32)(NumpyIO)




Support us on Patreon: https://www.patreon.com/HudsonThames

MlFinLab needs you! We need your help for us to keep on maintaining and implementing academic research based on 
financial machine learning (for open-source). In order for us to continue we need to raise $4000 of monthly donations
via Patreon - by December 2020. If we can't reach our goal, we will need to adopt more of a paid for service. We thought
that the best and least impactful course of action (should we not reach our goal) is to leave the package as open-source
but to make the documentation (ReadTheDocs) a paid for service. This is the ultimate litmus test, if the package is a 
value add, then we need the community to help us keep it going.

Our road map for 2020 is to implement the text book: Machine Learning for Asset Managers by Marcos Lopez de Prado, 
as well as a few papers from the Journal of Financial Data Science. We are hiring a full time developer for 3 months 
to help us reach our goals. The money that you

In [2]:
def save_bars(df, filename):
    if (os.path.exists(ODIR + filename)):
        fastparquet.write(ODIR + filename, df, append=True)
    else:
        fastparquet.write(ODIR + filename, df)

### Compute the number of bars of each type

Use dask to deal with 6 GB of tick data.

In [3]:
df = dd.read_parquet(IDIR + 'Y*.parq')
df

,price,volume
npartitions=26,,
,float64,float64
,...,...
...,...,...
,...,...
,...,...


In [4]:
total_len = len(df)
total_volume = df.volume.sum().compute()
total_dollar_volume = (df.volume * df.price).sum().compute()

In [5]:
total_len

261120215

In [6]:
total_volume

182117106.96479723

In [7]:
total_dollar_volume

1140035390521.672

In [8]:
time_bars = dd.read_parquet(ODIR + 'time.parq')
time_bars_len = len(time_bars)

In [9]:
ticks_per_bar = round(total_len / time_bars_len)
volume_per_bar = round(total_volume / time_bars_len)
dollars_per_bar = round(total_dollar_volume / time_bars_len)

In [10]:
ticks_per_bar

6411

In [11]:
volume_per_bar

4471.0

In [12]:
dollars_per_bar

27988005.0

In [13]:
files = sorted(glob.glob(IDIR + 'Y*.csv'))
files

['data/clean_tick/Y2017.csv',
 'data/clean_tick/Y2018_1.csv',
 'data/clean_tick/Y2018_2.csv',
 'data/clean_tick/Y2018_3.csv',
 'data/clean_tick/Y2019_1.csv']

# Tick bars

In [14]:
for file in tqdm(files):
    tick_bars = ml.data_structures.get_tick_bars(file, 
                                                 threshold=ticks_per_bar, 
                                                 batch_size=1000000)
    tick_bars.set_index('date_time', inplace=True)
    save_bars(tick_bars, 'tick.parq')

Reading data in batches:
Batch number: 0
Batch number: 1
Batch number: 2
Batch number: 3
Batch number: 4
Batch number: 5
Batch number: 6
Batch number: 7
Batch number: 8
Batch number: 9
Batch number: 10
Batch number: 11
Batch number: 12
Batch number: 13
Batch number: 14
Batch number: 15
Batch number: 16
Batch number: 17
Batch number: 18
Batch number: 19
Batch number: 20
Batch number: 21
Batch number: 22
Batch number: 23
Batch number: 24
Batch number: 25
Batch number: 26
Batch number: 27
Batch number: 28
Batch number: 29
Batch number: 30
Batch number: 31
Batch number: 32
Batch number: 33
Batch number: 34
Batch number: 35
Batch number: 36
Batch number: 37
Batch number: 38
Batch number: 39
Batch number: 40
Batch number: 41
Batch number: 42
Returning bars 

Reading data in batches:
Batch number: 0
Batch number: 1
Batch number: 2
Batch number: 3
Batch number: 4
Batch number: 5
Batch number: 6
Batch number: 7
Batch number: 8
Batch number: 9
Batch number: 10
Batch number: 11
Batch number: 12
B

In [15]:
def fix_duplicated_indices(df):
    delta = min(100, df.index.duplicated().sum())
    df.reset_index(inplace=True)
    
    for idx in range(1, len(df)):
        if df.loc[idx, 'date_time'] == df.loc[idx-1, 'date_time']:
            df.loc[idx-1, 'date_time'] = df.loc[idx-1, 'date_time'] - pd.Timedelta(microseconds = delta)
            delta -= 1
        else:
            delta = 100

    assert df['date_time'].duplicated().sum() == 0

# Volume bars

In [16]:
for file in tqdm(files):
    volume_bars = ml.data_structures.get_volume_bars(file, 
                                                     threshold=volume_per_bar, 
                                                     batch_size=1000000)
    volume_bars.set_index('date_time', inplace=True)
    save_bars(volume_bars, 'volume.parq')

Reading data in batches:
Batch number: 0
Batch number: 1
Batch number: 2
Batch number: 3
Batch number: 4
Batch number: 5
Batch number: 6
Batch number: 7
Batch number: 8
Batch number: 9
Batch number: 10
Batch number: 11
Batch number: 12
Batch number: 13
Batch number: 14
Batch number: 15
Batch number: 16
Batch number: 17
Batch number: 18
Batch number: 19
Batch number: 20
Batch number: 21
Batch number: 22
Batch number: 23
Batch number: 24
Batch number: 25
Batch number: 26
Batch number: 27
Batch number: 28
Batch number: 29
Batch number: 30
Batch number: 31
Batch number: 32
Batch number: 33
Batch number: 34
Batch number: 35
Batch number: 36
Batch number: 37
Batch number: 38
Batch number: 39
Batch number: 40
Batch number: 41
Batch number: 42
Returning bars 

Reading data in batches:
Batch number: 0
Batch number: 1
Batch number: 2
Batch number: 3
Batch number: 4
Batch number: 5
Batch number: 6
Batch number: 7
Batch number: 8
Batch number: 9
Batch number: 10
Batch number: 11
Batch number: 12
B

# Dollar bars

In [17]:
for file in tqdm(files):
    dollar_bars = ml.data_structures.get_dollar_bars(file, 
                                                     threshold=dollars_per_bar, 
                                                     batch_size=1000000)
    dollar_bars.set_index('date_time', inplace=True)
    save_bars(dollar_bars, 'dollar.parq')

Reading data in batches:
Batch number: 0
Batch number: 1
Batch number: 2
Batch number: 3
Batch number: 4
Batch number: 5
Batch number: 6
Batch number: 7
Batch number: 8
Batch number: 9
Batch number: 10
Batch number: 11
Batch number: 12
Batch number: 13
Batch number: 14
Batch number: 15
Batch number: 16
Batch number: 17
Batch number: 18
Batch number: 19
Batch number: 20
Batch number: 21
Batch number: 22
Batch number: 23
Batch number: 24
Batch number: 25
Batch number: 26
Batch number: 27
Batch number: 28
Batch number: 29
Batch number: 30
Batch number: 31
Batch number: 32
Batch number: 33
Batch number: 34
Batch number: 35
Batch number: 36
Batch number: 37
Batch number: 38
Batch number: 39
Batch number: 40
Batch number: 41
Batch number: 42
Returning bars 

Reading data in batches:
Batch number: 0
Batch number: 1
Batch number: 2
Batch number: 3
Batch number: 4
Batch number: 5
Batch number: 6
Batch number: 7
Batch number: 8
Batch number: 9
Batch number: 10
Batch number: 11
Batch number: 12
B